**Необходимо знать, что интересует пользователей в связи с оскаром: до него, во время и после. Для этого вам предлагается проанализировать логи поисковых запросов. Далее предложите несколько вариантов классификации этих запросов.**

In [2]:
import pandas as pd
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from pymystem3 import Mystem
import numpy as np

**Считаем логи поисковых запросов. Посмотрим на данные и время, за которое они собраны.**

In [15]:
oscar = pd.read_csv('oscar.csv', delimiter='\t')
oscar.head()

,normal_query,datetime
0,neityfz говядина как консервы,2019-01-23 09:39:25
1,водокаал алексин,2019-01-23 09:39:25
2,настя шевцова и гуф,2019-01-23 09:39:25
3,ул барсова 7 сочи,2019-01-23 09:39:25
4,доп соглашение к срочному трудовому договору о...,2019-01-23 09:39:25


In [3]:
print(min(oscar['datetime']), max(oscar['datetime']))

2019-01-22 00:00:00 2019-02-28 23:59:58


**Присвоим переменным start_dttm и end_dttm время начала и окончания премии Оскар. Добавим новый столбец, отображающий когда был сделан запрос: до премии, во время нее или после.**

In [4]:
start_dttm = '2019-01-25 04:00:00'
end_dttm = '2019-01-25 07:30:00'

def period_query (date):
    if date < start_dttm:
        return 'before'
    elif date <= end_dttm:
        return 'during'
    else:
        return 'after'
    
oscar['time_period'] = oscar['datetime'].apply(period_query)
oscar.head()

,normal_query,datetime,time_period
0,neityfz говядина как консервы,2019-01-23 09:39:25,before
1,водокаал алексин,2019-01-23 09:39:25,before
2,настя шевцова и гуф,2019-01-23 09:39:25,before
3,ул барсова 7 сочи,2019-01-23 09:39:25,before
4,доп соглашение к срочному трудовому договору о...,2019-01-23 09:39:25,before


**Перед классификацией необходимо провести предобработку. Для этого приведем запросы к нижнему регистру и разобьем их на токены. Также удалим стоп слова, пунктуацию и приведем слова к начальной форме (нормализуем).**

In [5]:
stop_words = set(stopwords.words('russian')) 
punctuation_set = set(punctuation)
mystem = Mystem()

In [61]:
%%time
def preprocess(query):
    tokenize = mystem.lemmatize(str(query).strip().lower())
    tokenize.pop()
    clean_queries = [
        word for word in tokenize if word not in stop_words and word not in punctuation_set
    ]
    return "".join(clean_queries)

oscar['normalize_word'] = oscar['normal_query'].apply(preprocess)

CPU times: user 25min 23s, sys: 6min 30s, total: 31min 54s
Wall time: 2h 20min 55s


In [66]:
oscar.head(10)

,normal_query,datetime,time_period,normalize_word
0,neityfz говядина как консервы,2019-01-23 09:39:25,before,neityfz говядина консервы
1,водокаал алексин,2019-01-23 09:39:25,before,водокаал алексин
2,настя шевцова и гуф,2019-01-23 09:39:25,before,настя шевцова гуф
3,ул барсова 7 сочи,2019-01-23 09:39:25,before,ул барсов 7 сочи
4,доп соглашение к срочному трудовому договору о...,2019-01-23 09:39:25,before,допы соглашение срочный трудовой договор пер...
5,carku e power 21,2019-01-23 09:39:25,before,carku e power 21
6,ловим налима осенью видео,2019-01-23 09:39:25,before,ловить налим осень видео
7,росгосстрах,2019-01-23 09:39:25,before,росгосстрах
8,конвертер валют,2019-01-23 09:39:25,before,конвертер валюта
9,карточка на ручку номера не беспокоить купить ...,2019-01-23 09:39:25,before,карточка ручка номер беспокоить купить крас...


In [65]:
oscar.to_csv('norm_oscar.csv', encoding='utf-8', index=False)

In [3]:
oscar = pd.read_csv('norm_oscar.csv')
oscar.head()

,normal_query,datetime,time_period,normalize_word
0,neityfz говядина как консервы,2019-01-23 09:39:25,before,neityfz говядина консервы
1,водокаал алексин,2019-01-23 09:39:25,before,водокаал алексин
2,настя шевцова и гуф,2019-01-23 09:39:25,before,настя шевцова гуф
3,ул барсова 7 сочи,2019-01-23 09:39:25,before,ул барсов 7 сочи
4,доп соглашение к срочному трудовому договору о...,2019-01-23 09:39:25,before,допы соглашение срочный трудовой договор пер...


**Разделим датасет на запросы: до Оскара, во время и после него.**

In [5]:
before_oscar = oscar[oscar['time_period'] =='before']['normalize_word'].dropna()
during_oscar = oscar[oscar['time_period'] =='during']['normalize_word'].dropna()
after_oscar = oscar[oscar['time_period'] =='after']['normalize_word'].dropna()

**Создадим функцию, которая делает tf-idf преобразование. Оно позволяет учитывать не только наличие слова в запросе, но и его значимость.**

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
tfidf = TfidfVectorizer(min_df=10) #токены, имеющие частотноть ниже 10 не будут учитываться

def tf_idf(data):
    vec_tfidf = tfidf.fit_transform(data)
    return pd.DataFrame(data=vec_tfidf.toarray(), columns=tfidf.get_feature_names(), index=data)

Посмотрим, что интересует пользователей до Оскара. Выберем 50 000 рандомных запросов и кластеризуем их.

In [7]:
from random import sample 

sample1 = sample(list(before_oscar), 50000)

**Воспользуемся методом k-средних для кластеризации.**

In [10]:
from sklearn.cluster import KMeans

#Ищем группы.
k_means = KMeans(n_clusters=20,
                init='k-means++',
                n_init=10,
                max_iter=300,
                tol=0.0001,
                n_jobs=5)

k_means.fit(tf_idf(sample1))

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=20, n_init=10, n_jobs=5, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [19]:
#Ключевые слова выводим в группы для анализа
dct = {}
for key, label in zip(sample1, k_means.labels_):
    dct[label] = dct.get(label, []) + [key]

In [28]:
for x in range(20):
    for line in dct[x]:
        if 'оскар' in line or 'oscar' in line:
            print(x)
            print(line)

1
тушь фаберлик ваш оскар
1
оскар синеть
1
список номинант  оскар 2019
1
статуэтка оскар купить омск
1
  телевизионный канал  россия  посмотреть награждение премия оскар
1
оскар 2019
1
оскар 2019
1
фоскарнет
1
тес роскар
1
оскар ремез  бабушка яга
17
фильм  оскар 2019


**Создадим Sample 2 и посмотрим насколько сильные различия в кластеризации разных выборок.**

In [32]:
sample2 = sample(list(before_oscar), 50000)
del before_oscar

In [33]:
k_means = KMeans(n_clusters=100,
                init='k-means++',
                n_init=20,
                max_iter=300,
                n_jobs=5)

k_means.fit(tf_idf(sample2))

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=100, n_init=20, n_jobs=5, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [34]:
dct = {}
for key, label in zip(sample2, k_means.labels_):
    dct[label] = dct.get(label, []) + [key]

In [36]:
for x in range(100):
    for line in dct[x]:
        if 'оскар' in line or 'oscar' in line:
            print(x)
            print(line)

2
номинанта  оскар
35
фильм награждать премия оскар  хороший
44
слушать оскар    
70
оскар     скачать бесплатно mp3
86
оскар 2019 номинанта
86
номинанта  оскар 2019
86
номинанта  оскар 2019 список


Несмотря на то, что имеются различия в двух выборках, по кластерам обеих выборок можно сделать вывод. Больше всего пользователей до начала премии Оскар интересует список номинантов и возможность посмотреть трансляцию премии.

Теперь посмотрим, что интересует пользователей во время Оскара.

In [31]:
during_oscar = oscar[oscar['time_period'] =='during']['normalize_word'].dropna()
len(during_oscar)

57805

In [32]:
#Прерывала после повторного запуска -- долгоооо
k_means = KMeans(n_clusters=10,
                init='k-means++',
                n_init=20,
                max_iter=300,
                n_jobs=5)

k_means.fit(tf_idf(during_oscar))

KeyboardInterrupt: 

In [28]:
dct = {}
for key, label in zip(during_oscar, k_means.labels_):
    dct[label] = dct.get(label, []) + [key]

In [29]:
for x in range(10):
    for line in dct[x]:
        if 'оскар' in line or 'oscar' in line:
            print(x)
            print(line)

4
оскар
4
премия оскар
4
читать мотивационный книга  книга оскар уалд
4
патриотизм религия бешеный оскар уайльд  английский
4
греф герман оскарович биография


Во время премии Оскар пользователей интересует премия Оскар!

Посмотрим, что интересует пользователей после Оскара.

In [ ]:
sample3 = sample(list(after_oscar), 30000)

k_means = KMeans(n_clusters=10,
                init='k-means++',
                n_init=20,
                max_iter=300,
                n_jobs=5)

k_means.fit(tf_idf(sample3))

In [ ]:
dct = {}
for key, label in zip(sample3, k_means.labels_):
    dct[label] = dct.get(label, []) + [key]

In [ ]:
for x in range(10):
    for line in dct[x]:
        if 'оскар' in line or 'oscar' in line:
            print(x)
            print(line)